In [6]:
from Agent import Agent

agent_name = "OpenAI"
agent = Agent(agent_name=agent_name)
agent_config = agent.agent_config


2023-05-31 14:12:21,424 | INFO | loaded in 1 embeddings
2023-05-31 14:12:21,425 | INFO | loaded in 1 collections
2023-05-31 14:12:21,425 | INFO | Persisting DB to disk, putting it in the save folder: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories


In [7]:
collection = await agent.memories.get_collection()

In [4]:
await agent.memories.store_memory(
    content="Test Memory", description="Test Memory", external_source_name="Test Memory"
)

2023-05-31 14:12:13,158 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=323 request_id=b9ac567f398ba8a5aa06179083deaf9a response_code=200
2023-05-31 14:12:13,169 | INFO | Persisting DB to disk, putting it in the save folder: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories
2023-05-31 14:12:13,182 | INFO | Persisting DB to disk, putting it in the save folder: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories


In [8]:
mems = await agent.memories.context_agent(query="Test Memory", top_results_num=1)

2023-05-31 14:12:26,605 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=308 request_id=26065e04fb5fb09b5e984325b77c666b response_code=200
2023-05-31 14:12:26,858 | INFO | CONTEXT: ['Test Memory']


In [17]:
# Create a new memory
import os
from hashlib import sha256
from datetime import datetime
from semantic_kernel.memory.memory_record import MemoryRecord
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore
from chromadb.config import Settings
from Agent import Agent

agent_name = "OpenAI"
agent = Agent(agent_name=agent_name)
agent_config = agent.agent_config
embedder, chunk_size = await agent.get_embedder()
memories_dir = os.path.join(os.getcwd(), "agents", agent_name, "memories")
chroma_client = ChromaMemoryStore(
    persist_directory=memories_dir,
    client_settings=Settings(
        chroma_db_impl="chromadb.db.duckdb.PersistentDuckDB",  # also tried 'duckdb+parquet' but it did not work at all.
        persist_directory=memories_dir,
        anonymized_telemetry=False,
    ),
)


async def get_collection():
    memories_exist = await chroma_client.does_collection_exist_async("memories")
    print(f"Memories Exist? {memories_exist}")
    if not memories_exist:
        print("Creating memories collection")
        await chroma_client.create_collection_async(collection_name="memories")
        memories = await chroma_client.get_collection_async(collection_name="memories")
    else:
        memories = await chroma_client.get_collection_async(collection_name="memories")
        print(
            "Retrieved memories collection"
        )  # <- This never happens on the first attempt
    return memories


async def new_memory(
    content="Test memory", description="Test memory", external_source_name="Test memory"
):
    await get_collection()
    record = MemoryRecord(
        is_reference=False,
        id=sha256((content + datetime.now().isoformat()).encode()).hexdigest(),
        text=content,
        timestamp=datetime.now().isoformat(),
        description=description,
        external_source_name=external_source_name,  # URL or File path
        embedding=await embedder(content),
    )

    await chroma_client.upsert_async(
        collection_name="memories",
        record=record,
    )


async def get_memory(content="Test memory"):
    # Find a memory
    results = await chroma_client.get_nearest_matches_async(
        collection_name="memories",
        embedding=await embedder(content),
        limit=1,
        min_relevance_score=0.1,
    )
    context = []
    for memory, score in results:
        context.append(memory._text)
    return context


async def memories_exist():
    return await chroma_client.does_collection_exist_async("memories")


2023-05-31 10:24:28,447 | INFO | loaded in 0 embeddings
2023-05-31 10:24:28,448 | INFO | loaded in 0 collections
2023-05-31 10:24:28,448 | INFO | Persisting DB to disk, putting it in the save folder: /home/josh/josh/Repos/AGiXT/agixt/agents/OpenAI/memories


In [20]:
await memories_exist()

True

In [19]:
await get_collection()


Memories Exist? False
Creating memories collection


Collection(name=memories)

In [21]:
await new_memory(
    description="Test memory", external_source_name="Test memory", content="Test memory"
)


Memories Exist? True
Retrieved memories collection


2023-05-31 10:24:35,748 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=350 request_id=3f271c44603a8efae2fa97eb4742e2ea response_code=200


In [22]:
memory = await get_memory(content="Test memory")
print(memory)

2023-05-31 10:24:38,273 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=145 request_id=34ebe3041bbe1b6770cb8d77960fb499 response_code=200


['Test memory']
